In [78]:
import glob
import os
import re
import datetime
import pandas as pd

def get_stationary_asset(text):
    asset = []
    data = {
        'first': 'Dendukuri House',
        'second': 'Galaxy Aurobindo 9th Floor',
        'third': 'Galaxy Aurobindo 5th Floor',
    }
    for line in text:
        if re.search('.*7th.*floor.*block.*', line.lower()) or re.search('.*9th.*floor.*galaxy.*', line.lower()) or re.search('.*aurobindo.*galaxy.*9th.*floor.*', line.lower()):
            asset.append(data['second'])
            break
        if re.search('.*plot.*no.*564.*a.*39.*', line.lower()):
            asset.append(data['first'])
            break
    return asset

def get_traveler_name(text):
    traveler_name = []
    flag = 0
    for line in text:
        if 'name' in line.lower() and 'bill' in line.lower():
            words = line.split()
            lowerwords = line.lower().split()
            for i in range(len(words)):
                if 'name' in words[i].lower():
                    for j in range(len(lowerwords)):
                        if 'bill' in lowerwords[j]:
                            ind = j
                            break
                    flag = 1
                    traveler_name.append(' '.join(words[i+2:ind]))
                    # print('NAME--------------> ', traveler_name)
                    break
            if flag:
                break

    return traveler_name

def get_start_date(text):
    date = ''
    new_date = []
    flag = 0
    for line in text:
        if 'arrival date' in line.lower():
            words = line.split()
            lwords = line.lower().split()
            ind = lwords.index('date')
            for i in range(ind+1, len(words)):
                # print('WORDS---> ', words[i])
                if '/' in words[i]:
                    flag = 1
                    date = words[i]
                    break
            if flag:
                break
    
    formats = ['%d/%m/%Y']
    for format in formats:
        try:
            d = datetime.datetime.strptime(date, format).strftime('%Y-%m-%d')
            new_date.append(d)
        except:
            pass
    
    return new_date

def get_end_date(text):
    date = ''
    new_date = []
    flag = 0
    for line in text:
        if 'departure date' in line.lower():
            words = line.split()
            lwords = line.lower().split()
            if 'date' in lwords:
                ind = lwords.index('date')
            if 'date:' in lwords:
                ind = lwords.index('date:')
            for i in range(ind+1, len(words)):
                # print('WORDS---> ', words[i])
                if '/' in words[i]:
                    flag = 1
                    date = words[i]
                    break
            if flag:
                break
    
    formats = ['%d/%m/%Y']
    for format in formats:
        try:
            d = datetime.datetime.strptime(date, format).strftime('%Y-%m-%d')
            new_date.append(d)
        except:
            pass
    
    return new_date

def get_no_of_rooms(text):
    rooms = []
    no_of_rooms = []
    for line in text:
        if 'room no' in line.lower():
            words = line.split()
            lwords = line.lower().split()
            ind = lwords.index('room')
            for i in range(ind, len(words)):
                if words[i].isnumeric():
                    rooms.append(words[i])
            break
    no_of_rooms.append(len(rooms))
    return no_of_rooms

def get_no_of_nights(text):
    no_of_nights = []
    nights = 0
    try:
        start_date = get_start_date(text)
        end_date = get_end_date(text)
        for i in range(len(start_date)):
            fyear, fmonth, fday = start_date[i].split('-')
            fdate = datetime.date(int(fyear), int(fmonth), int(fday))
            lyear, lmonth, lday = end_date[i].split('-')
            ldate = datetime.date(int(lyear), int(lmonth), int(lday))
            nights = (ldate-fdate).days
            # print('NIGHTS----> ', nights)
            no_of_nights.append(nights)
        return no_of_nights
    except:
        pass

def get_hotel_name(text):
    hotel_name = []
    for i in range(len(text)):
        if 'hotels' in text[i].lower() and len(text[i].split()) > 1:
            hotel_name.append(text[i])
            break
        if 'tax invoice' in text[i].lower():
            hotel_name.append(text[i+1])
            break
    return hotel_name

def clean_name(hotel_city):
    final_hotel_city = []
    for hc in hotel_city:
        if ',' in hc:
            hc = hc.replace(',', '')
        if '-' in hc:
            hc = hc.replace('-', '')
        final_hotel_city.append(hc)
    if final_hotel_city:
        return final_hotel_city
    return hotel_city

def get_hotel_city(text):
    hotel_city = []
    for i in range(len(text)):
        try:
            if re.search('.*\d{6}.*', text[i]):
                words = text[i].split()
                print('WORDS---> ', words)
                if len(words) < 3 and 'pan no' not in text[i-1].lower():
                    print('TEXT I-1---> ', text[i-1])
                    hotel_city.append(text[i-1].split()[-2])
                    print(hotel_city)
                    break
                else:
                    words = text[i].split()
                    for j in range(len(words)):
                        if re.search('.*\d{6}.*', words[j]) and 'gstn' not in text[i].lower():
                            ind = j
                            break
                    if 'enclave' in text[i].lower():
                        hotel_city.append(words[ind-2])
                    else:
                        hotel_city.append(words[ind-1])
                    break
        except Exception as e:
            print('ERROR---> ', e)
            pass
    
    hotel_city = clean_name(hotel_city)
    return hotel_city

def get_hotel_country(text, data):
    hotel_country = []
    h_country = ''
    # hotel_city = get_hotel_city(text)
    
    # if hotel_city:
    #     for city in hotel_city:
    #         if city:
    #             try:
    #                 value = data[data['city'] == city.strip()].iloc[0]
    #                 h_country = value['country']
    #                 hotel_country.append(h_country)
    #                 return hotel_country
    #             except:
    #                 pass
    return ['IN']

files = []

# data = pd.read_excel('C:\\air_ticket\\World_DB.xlsx')

for file in glob.glob('C:\\air_ticket\\comakeit_hotel_stay\\ensemble_files\\*.txt'):
    print(file)
    
    filename = os.path.split(file)[-1]
    words = filename.split('_')
    # print(words)
    new_filename = ''
    for word in words:
        if 'page' in word:
            # print('YYEEESS')
            break
        new_filename += word+'_'
    # print(new_filename.strip('_'))

    stationary_asset_final = []
    traveler_name_final = []
    start_date_final = []
    end_date_final = []
    no_of_rooms_final = []
    no_of_nights_final = []
    hotel_name_final = []
    hotel_city_final = []
    hotel_country_final = []
    
    with open(file) as fp:

        texts = fp.read().split("---\n")
        # pprint(texts)
        for text in texts:
            text = text.splitlines()
            text = [line for line in text if not line.isspace() and len(line) > 0]
            # pprint(text)
            # filename = os.path.split(file)[-1].split('.txt')[0]

            # if new_filename not in files:

            stationary_asset = get_stationary_asset(text)
            if stationary_asset and not stationary_asset_final:
                stationary_asset_final = stationary_asset
            
            traveler_name = get_traveler_name(text)
            if traveler_name and not traveler_name_final:
                traveler_name_final = traveler_name

            start_date = get_start_date(text)
            if start_date and not start_date_final:
                start_date_final = start_date

            end_date = get_end_date(text)
            if end_date and not end_date_final:
                end_date_final = end_date
            
            no_of_rooms = get_no_of_rooms(text)
            if no_of_rooms and not no_of_rooms_final:
                no_of_rooms_final = no_of_rooms

            no_of_nights = get_no_of_nights(text)
            if no_of_nights and not no_of_nights_final:
                no_of_nights_final = no_of_nights

            hotel_name = get_hotel_name(text)
            if hotel_name and not hotel_name_final:
                hotel_name_final = hotel_name
            
            hotel_city = get_hotel_city(text)
            if hotel_city and not hotel_city_final:
                hotel_city_final = hotel_city

            hotel_country = get_hotel_country(text, data)
            if hotel_country and not hotel_country_final:
                hotel_country_final = hotel_country

        print('Stationary Asset---> ', stationary_asset_final)
        print('Traveler Name---> ', traveler_name_final)
        print('Start Date---> ', start_date_final)
        print('End Date---> ', end_date_final)
        print('Number of rooms---> ', no_of_rooms_final)
        print('Number of nights---> ', no_of_nights_final)
        print('Hotel Name---> ', hotel_name_final)
        print('Hotel City---> ', hotel_city_final)
        print('Hotel Country---> ', hotel_country_final)
        print('----------------------')

        files.append(new_filename)

C:\air_ticket\comakeit_hotel_stay\ensemble_files\Avasahotels_BQ2901BIL0000097_06.05.2022_page0.txt
WORDS--->  ['Enclave,Madhapur,', ',', 'Hyderabad,', 'Telangana,', '500081']
WORDS--->  ['Enclave,Madhapur,', ',', 'Hyderabad,', 'Telangana,', '500081']
Stationary Asset--->  ['Galaxy Aurobindo 9th Floor']
Traveler Name--->  []
Start Date--->  []
End Date--->  []
Number of rooms--->  [0]
Number of nights--->  []
Hotel Name--->  ['AVASA HOTELS (A Unit of Stamlo Industries Limited)']
Hotel City--->  ['Hyderabad']
Hotel Country--->  ['IN']
----------------------
C:\air_ticket\comakeit_hotel_stay\ensemble_files\avasahotels_FM2901BIL0000479_08.04.2022_page0.txt
WORDS--->  ['Madhapur,', 'Hyderabad-', '500081', 'T', ':', '91-', '040', '—', '67', '28', '28', '28']
WORDS--->  ['Madhapur,', 'Hyderabad-', '500081', 'T', ':', '91-', '040', '—', '67', '28', '28', '28']
Stationary Asset--->  ['Galaxy Aurobindo 9th Floor']
Traveler Name--->  []
Start Date--->  ['2022-03-30']
End Date--->  ['2022-04-08']
